# Download and Process BLS QCEW files

https://www.bls.gov/cew/additional-resources/open-data/csv-data-slices.htm#INDUSTRY-SLICES

### Download data files

- NAICS 311 quarterly county establishments and employment (e.g. https://data.bls.gov/cew/data/api/2024/4/industry/311.csv)
- [Documentation](https://www.bls.gov/cew/classifications/)


In [2]:
import os
import requests
import pandas as pd

DOWNLOAD_DIR = "downloads"

In [ ]:
# function to download BLS QCEW csvs
def download_file(url, year, qtr, save_directory=DOWNLOAD_DIR):
    """
    Downloads a file from a given URL and saves it to a specified directory.
    """
    try:
        # Create the save directory if it doesn't exist
        os.makedirs(save_directory, exist_ok=True)

        # Extract the filename from the URL
        filename = os.path.basename(url).split('.')[0]
        save_path = os.path.join(save_directory, f'{filename}_{year}_{qtr}.csv')

        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"File downloaded successfully: {save_path}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
# download quarterly files for range of years
startyear = 2014
endyear = 2023

for year in range(startyear,endyear+1):
    for qtr in range(1,5):
        file_url = fr"https://data.bls.gov/cew/data/api/{year}/{qtr}/industry/311.csv"
        download_file(file_url, year, qtr)

### Process data files

In [2]:
# read in each csv as a dataframe and add to list
n_columns = 9
all_dataframes = []

for filename in os.listdir(DOWNLOAD_DIR):
    if filename.endswith('.csv'):
        filepath = os.path.join(DOWNLOAD_DIR, filename)
        try:
            df = pd.read_csv(filepath)
            df_subset = df.iloc[:, :n_columns]
            all_dataframes.append(df_subset)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

In [3]:
# merge csvs into a single df
df = pd.concat(all_dataframes, ignore_index=True)

In [4]:
# filter based on fips code
filtered_df = df[
    ~(
        # filter out state records (xx000) and unknown/undefined areas (xx999)
        df.area_fips.astype(str).str.endswith(('000', '999')) |
        # filter out PR (72xxx) and USVI (78xxx)
        df.area_fips.astype(str).str.startswith(('72', '78')) |
        # filter out MSAs and CSAs
        df.area_fips.astype(str).str.match(r'^[A-Za-z]')
    )
]

# filtered out ownership codes other than 5 (govt, etc.), leaving just private businesses
filtered_df = filtered_df[filtered_df.own_code==5]

# add date field based on year and quarter
filtered_df['date'] = pd.to_datetime(filtered_df['year'].astype(str) + 'Q' + filtered_df['qtr'].astype(str))

C:\Users\mar10556\AppData\Local\Temp\1\ipykernel_42880\1251876053.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_df['date'] = pd.to_datetime(filtered_df['year'].astype(str) + 'Q' + filtered_df['qtr'].astype(str))


In [ ]:
# write out to csv
filtered_df.to_csv('BLS_QCEW_311_2014_2023.csv')